In [46]:
import pandas as pd
import numpy as np

In [137]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel


from bokeh.palettes import Category20_16

output_notebook()

Loading BokehJS ...

# Reading in Data

In [309]:
# load in params and websites
webs_paras = pd.read_csv('./data/normized.csv')

In [310]:
webs = webs_paras.drop(['(Doc complete) Byets in','(Fully loaded) Bytes in','(Fully loaded) Requests','(Fully loaded) Requests','(Doc complete) Requests'], axis = 1)

In [311]:
webs_requests = webs_paras[['Site name','(Fully loaded) Requests','(Fully loaded) Requests','(Doc complete) Requests']]

In [312]:
#webs

In [313]:
 #webs['(Doc complete) Byets in'].describe()

In [314]:
list_of_params = list(webs['Site name'].unique())

In [315]:
list_of_params.sort()

In [316]:
len(list_of_params) 

49

In [317]:
#for i in range(49):
    #print i%10

In [318]:
Category20_16[0]

'#1f77b4'

# Function to Make Dataset for plot


In [319]:
def make_dataset(params_list, range_start = 1, range_end = 50, bin_width = 5):
    
    #check to make sure the start is less than the end
    assert range_start < range_end, "Start must be less than end!"
    
    by_params = pd.DataFrame(columns=[ 'left','right', 'p_interval', 'name', 'color']) 
   # 'proportion', 'p_proportion'
    
    range_extent = range_end - range_start
    
    # Iterate through all the parameters 
    for i, para_name in enumerate(params_list):
    
        
        # Subset to the parameter
        subset = webs[webs['Site name'] == para_name]
        
        
        
        
        # Create a histogram with specified bins and range
        arr_hist, edges = np.histogram(subset[webs.columns[i%6]], 
                                      bins = int(range_extent / bin_width),
                                      range = [range_start, range_end])
        
        print arr_hist
        
        # Divide the counts by the total to get a proportion and create df
        arr_df= pd.DataFrame({
                              'left': edges[:-1], 'right': edges[1:]}) #'proportion': arr_hist / np.sum(arr_hist),
      
        # Format the proportion
        #arr_df['p_proportion'] = ['%0.5f' % proportion for proportion in arr_df['proportion']]
        
        # Format the interval
        arr_df['p_interval'] = ['%d to %d scale' % (left, right) for left, 
                               right in zip(arr_df['left'], arr_df['right'])]
        
        # Assign the parameter for labels
        arr_df['name'] = para_name
        
        # Color each parametr differently
        arr_df['color'] = Category20_16[i%16]
        
        # Add to the overall dataframe
        by_params = by_params.append(arr_df)
        
    # Overall dataframe
    by_params = by_params.sort_values(['name','left'])
    
    
    
    return by_params
        
        
        
        

In [320]:
#make_dataset(list_of_params).head()
p=make_plot(ColumnDataSource(make_dataset(list_of_params)))
show(p)

[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]


# Function to Make Plot from Data Source

In [300]:
def style(p):
    # Title
    p.title.align = 'center'
    p.title.text_font_size ='20pt'
    p.title.text_font = 'serif'
    
    # Axis titles
    p.xaxis.axis_label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '14pt'
    p.yaxis.axis_label_text_font_style = 'bold'
    
    # Tick labels
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'
    
    return p

In [301]:
def make_plot(src):
    # Blank plot with correct labels
    p = figure(plot_width = 700, plot_height = 700,
              title = "Histogram of Parametes for the websites",
              x_axis_label = 'x_label', y_axis_label = "y_labe")
    
    # Quad glyphs to create a histogram
    p.quad(source=src, bottom =0,left = 'left', right = 'right', color ='color', top= 0.5,fill_alpha = 0.7, hover_fill_color = 'color', legend = 'name',
           hover_fill_alpha = 1.0, line_color = 'white') #top='proportion',
    
    # Hover tool with vline mode
    hover = HoverTool(tooltips=[('Parameter','@name'),
                               ('Difference','@p_interval')],
                     mode='vline')
    
    p.add_tools(hover)
    # Stypling
    p = style(p)
    return p